<a href="https://colab.research.google.com/github/jpiche0720/DS-Unit-2-Applied-Modeling/blob/master/module3-permutation-boosting/LS_DS_233_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 3*

---


# Permutation & Boosting

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] If you haven't completed assignment #1, please do so first.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline? 
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.

But, if you aren't ready to try xgboost and permutation importances with your dataset today, that's okay. You can practice with another dataset instead. You may choose any dataset you've worked with previously.

The data subdirectory includes the Titanic dataset for classification and the NYC apartments dataset for regression. You may want to choose one of these datasets, because example solutions will be available for each.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - _**[A Kaggle Master Explains Gradient Boosting](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/)**_
  - [_An Introduction to Statistical Learning_](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf) Chapter 8
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html)
  - _**[Boosting](https://www.youtube.com/watch?v=GM3CDQfQ4sw) (2.5 minute video)**_

# XGBOOST and Permutation Importances

In [0]:
import pandas as pd 
import numpy as np
from xgboost import XGBRegressor
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


In [0]:
energy = pd.read_csv('Project_final.csv')

In [0]:
train = energy.iloc[0:550,:]
val = energy.iloc[550:700,:]
test = energy.iloc[700:,:]
train.shape,val.shape,test.shape

((550, 38), (150, 38), (116, 38))

In [0]:
target = 'total natural gas consumption for electric power2002'
features = train.columns.drop(['state','All industry total','Private industries',target,'STATE','YEAR'])

In [0]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_true = test[target]

In [0]:
model = XGBRegressor(n_estimators=100,max_depth=10,random_state=42,n_jobs=-1)


model.fit(X_train,y_train)
y_pred = model.predict(X_val)
mean_absolute_error(y_val,y_pred)

[23:14:41] WARNING: /usr/local/miniconda/conda-bld/xgboost_1572315027083/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


40.226914157770466

In [0]:
permuter = PermutationImportance(model,
                                 scoring ='neg_mean_absolute_error',
                                 n_iter=5,
                                 random_state=42)

permuter.fit(X_val,y_val)

feature_names = X_val.columns.to_list()


eli5.show_weights(permuter,
                  top=None,
                  feature_names=feature_names)

Weight,Feature
20.0204 ± 3.1953,Accommodation and food services
19.1888 ± 10.7122,Construction
17.3749 ± 4.4498,Natural resources and mining
11.3826 ± 2.1182,Nondurable goods manufacturing
10.7918 ± 2.0440,Utilities
10.6586 ± 3.5839,population_estimate
9.1301 ± 3.0396,"Mining, quarrying, and oil and gas extraction"
8.6490 ± 1.5023,Professional and business services
6.2140 ± 1.6762,Manufacturing
3.7730 ± 2.2916,Management of companies and enterprises
